# Evaluate the classifier's transferability on a tiled dataset from Bern cohorts

In [1]:
import importlib
import os
import functools

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import ImageFolder
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from tqdm import tqdm

import Abed_utils

## Load and build model

In [7]:
patch_size = 8
batch_size = 8
im_size = 224
path_to_backbone = './ckpts/dino_deitsmall8_pretrain.pth'
path_to_classifier = './ckpts/classifier_K19_CE_100ep_200_100_hidden.pt'

transform = functools.partial(Abed_utils.normalize_input, im_size=im_size, patch_size=patch_size)

In [10]:
# backbone = Abed_utils.get_vit(patch_size, path_to_backbone)

classifier = Abed_utils.ClassificationHead(pretrained_path=path_to_classifier, dropout=0.2, n_hidden=2, hidden_dims=[200,100])
# features, labels = Abed_utils.load_features(os.path.join(Abed_utils.OUTPUT_ROOT, 'features'), cuda=True)
# classifier = Abed_utils.KNNClassifier(features, labels)
# model = nn.Sequential(backbone, classifier)
model = classifier
model.eval()

ClassificationHead(
  (mlp): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=384, out_features=200, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=100, out_features=9, bias=True)
  )
)

In [11]:
if Abed_utils.BERN_TILES_ROOT is None:
    raise RuntimeError('labelled Bern tiles not available on this machine, run on the dataserver')

wsis = os.listdir(Abed_utils.BERN_TILES_ROOT)
with torch.no_grad():
    for wsi in wsis:

        # ds = ImageFolder(os.path.join(Abed_utils.BERN_TILES_ROOT, wsi), transform=transform)
        features, labels = Abed_utils.load_features(os.path.join(Abed_utils.OUTPUT_ROOT, f'features-{wsi}'), cuda=True)
        ds = TensorDataset(features, labels)
        data = DataLoader(ds, batch_size=batch_size)
        preds = np.empty(len(ds))
        targets = np.empty_like(preds)
        for i, (x, y) in enumerate(tqdm(data)):
            idx = slice(i*batch_size,min((i+1)*batch_size, preds.shape[0]))
            predictions = model(x.cuda()).cpu().numpy()
            preds[idx] = predictions if len(predictions.shape) < 2 else predictions.argmax(axis=1)
            targets[idx] = y.cpu().numpy()

        acc = (preds == targets).mean()
        f1 = f1_score(targets, preds, average='micro')
        print(f'for {wsi} we get an accuracy of {acc:.4f} and an f1 of {f1:.4f}')


100%|██████████| 4006/4006 [00:03<00:00, 1002.68it/s]


for 001b_B2005.30530_C_HE.mrxs we get an accuracy of 0.6462 and an f1 of 0.6462


100%|██████████| 3322/3322 [00:03<00:00, 1048.70it/s]


for 352b_B2005.5775_F_HE.mrxs we get an accuracy of 0.5703 and an f1 of 0.5703


100%|██████████| 3362/3362 [00:03<00:00, 1023.74it/s]

for 565c_B2012.15587_B_HE.mrxs we get an accuracy of 0.5876 and an f1 of 0.5876
